In [725]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')



In [726]:
cproducts = pd.read_csv("data/cproducts.csv")
ctender = pd.read_csv("data/ctender.csv")

In [727]:
cproducts.loc[cproducts.customerID=='BBID_20482',['Gender','PinCode']]

,Gender,PinCode
0,male,453441.0


In [728]:
### For quick start dropping missing value
#cproducts.dropna(inplace=True)
#cproducts.DOB[cproducts.DOB=='NANA']=None  # Removing Noisy records

### Setting appropriate datatype to the field


In [729]:
cproducts['PinCode']=cproducts.PinCode.astype(int)
cproducts['DOB']=cproducts.DOB.astype("datetime64[ns]")
cproducts['Gender']=cproducts.Gender.astype('category')
cproducts['State']=cproducts.State.astype('category')
cproducts['transactionDate']=cproducts.transactionDate.astype('datetime64[ns]')
cproducts['store_code']=cproducts.store_code.astype('category')
cproducts['store_description']=cproducts.store_description.astype(str)
cproducts['till_no']=cproducts.till_no.astype('category')
cproducts['promo_code']=cproducts.promo_code.astype('category')
cproducts['product_code']=cproducts.product_code.astype('category')
cproducts['discountUsed']=cproducts.discountUsed.astype('category')

ValueError: Cannot convert non-finite values (NA or inf) to integer

In [730]:
### Usefull Field by Assumption
# DOB
# Gender
# State
# Pincode
# transactionDate
# Store Code
# Store Counter Number
# Promocode
# Product Code
# Sales Price After Promo
# Discount Used

In [731]:
cproducts.groupby(['Gender'])['Gender'].agg({'Freq':'count'}).reset_index()

,Gender,Freq
0,female,2953
1,male,7991


In [732]:
import requests
import json

def getState(x):
    r = requests.get("http://postalpincode.in/api/postoffice/%s"%str(x))
    data = json.loads(r.text)
    if(data['Status']=='Success'):
        return(data['PostOffice'][0]['State']).upper()
    else:
        return None
    
#getState('Bangalore') Sample Call

In [733]:
# If State and Pincode both are Null

t=cproducts.store_description[((cproducts.PinCode.isnull())&(cproducts.State.isnull()))]
t=t.str.split('-')
cproducts.State[((cproducts.PinCode.isnull())&(cproducts.State.isnull()))] = [ getState(w[1]) for w in t]


# If State is Null 

t=cproducts.store_description[((cproducts.State.isnull()))]
t=t.str.split('-')
res=[ getState(w[1]) for w in t]
cproducts.State[cproducts.State.isnull()]=res

# If State value has x

t=cproducts.store_description[cproducts.State=='x']
t=t.str.split('-')
cproducts.State[cproducts.State=='x']=[ getState(w[1]) for w in t]

# If State value has DUMMY

t=cproducts.store_description[cproducts.State=='DUMMY']
t=t.str.split('-')
cproducts.State[cproducts.State=='DUMMY']=[ getState(w[1]) for w in t]


In [735]:
## State Value cleaning 
cproducts['State']=cproducts.State.str.strip()
cproducts.State[cproducts.State=='TAMILNADU']="TAMIL NADU"
cproducts.State[cproducts.State=='CHATTISGARH']="CHHATTISGARH"
cproducts.State[cproducts.State=='HARAYANA']="HARYANA"
cproducts.State[cproducts.State=='MADHYA  PRADESH']="MADHYA PRADESH"
cproducts.State[cproducts.State=='MP']="MADHYA PRADESH"
cproducts.State[cproducts.State=='KARNATAK']="KARNATAKA"
cproducts.State[cproducts.State=='UTTAR PRADESH EAST']="UTTAR PRADESH"
cproducts.State[cproducts.State=='Jharkhand']="JHARKHAND"
cproducts.State[cproducts.State=='Other']="KARNATAKA" # Since it is Hubli

In [736]:
cproducts.groupby(['State'])['State'].agg({'Freq':'count'}).reset_index().sort_values(by='Freq',ascending=False)

,State,Freq
14,MADHYA PRADESH,3681
12,KARNATAKA,2733
11,JHARKHAND,1892
21,TAMIL NADU,1670
18,PUNJAB,1227
15,MAHARASHTRA,174
25,WEST BENGAL,106
23,UTTAR PRADESH,105
5,DELHI,60
2,BIHAR,49


In [737]:
cproducts.to_csv('data/Proccessed.csv')

In [552]:
cproducts = pd.read_csv("data/Proccessed.csv")

Unnamed: 0                       0
customerID                       0
DOB                            319
Gender                        1028
State                            0
transactionDate                  0
store_code                       0
till_no                          0
transaction_number_by_till       0
promo_code                       0
product_code                     0
product_description              0
sale_price_after_promo           0
discountUsed                     0
transactionDay                   0
Age                              0
dtype: int64

In [738]:
del cproducts['PinCode']
del cproducts['store_description']
del cproducts['promotion_description']


In [566]:
# Imputing Weekday Name from Transaction date
cproducts['transactionDay']=pd.to_datetime(cproducts.transactionDate,format='%Y-%m-%d').dt.weekday_name

In [697]:
import datetime
def getAge(x):
    A=pd.to_datetime(x)
    B=pd.to_datetime(datetime.datetime.now().strftime("%Y-%m-%d")) # Current Date
    return ((B-A).days//365)

#getAge('1993-03-11')

In [719]:
# Imputing Customer Age

cproducts["Age"]=-1
cproducts.Age[(cproducts.DOB.isnull()==False)&((cproducts.DOB=="NANA")==False)]=\
        [ getAge(w) for w in pd.to_datetime(cproducts.DOB[(cproducts.DOB.isnull()==False)&\
                                                  ((cproducts.DOB=="NANA")==False)]) ]

,Unnamed: 0,customerID,DOB,Gender,State,transactionDate,store_code,till_no,transaction_number_by_till,promo_code,product_code,product_description,sale_price_after_promo,discountUsed,transactionDay,Age
0,0,BBID_20482,1975-10-20,male,MADHYA PRADESH,2016-05-01,2655,2,85246,NONPROMO,1000446431,PATANJALI CHOCOS 125g,55.00,Payback,Sunday,42
1,1,BBID_20485,1955-07-21,female,MADHYA PRADESH,2017-03-09,2655,8,78829,NONPROMO,1000010431,SOO FRESH ROLLS HOT DOG 4P,30.00,Payback,Thursday,62
2,2,BBID_20498,1981-03-19,female,MAHARASHTRA,2015-08-12,2655,13,1064,0000971754,1000077851005,"DLJ-0010-BSF-2-CJ-FL, 34, BLACK",418.95,Payback,Wednesday,36
3,3,BBID_20499,1986-05-10,male,MAHARASHTRA,2016-12-18,2615,14,51080,NONPROMO,1000443231,PATANJALI TOMATO KETCHUP BT 500g,70.00,Payback,Sunday,31
4,4,BBID_204110,1988-02-29,male,MADHYA PRADESH,2015-12-31,2655,9,21911,0000974126,300481741,MAGGI NDL MERI MASALA PP 70g,34.20,Payback,Thursday,29


In [722]:
cproducts.isnull().sum()

Unnamed: 0                       0
customerID                       0
DOB                            319
Gender                        1028
State                            0
transactionDate                  0
store_code                       0
till_no                          0
transaction_number_by_till       0
promo_code                       0
product_code                     0
product_description              0
sale_price_after_promo           0
discountUsed                     0
transactionDay                   0
Age                              0
dtype: int64

In [723]:
cproducts.to_csv('data/Proccessed.csv')